# Data cleaning and basic analysis

In this session, we're going to apply what we've learned so far to some real-world use cases: Cleaning data and doing some basic analysis.

In many cases, it makes sense to use a data analysis library like [`pandas`](http://pandas.pydata.org/) or [`agate`](https://agate.readthedocs.io/en/1.6.0/). Our needs today are pretty simple, though, so we'll stick to Python's standard library.

Whatever tools you use, you need to do the reporting to understand the ways in which your data are flawed and develop a cleaning strategy. Then come up with a list of questions to ask your data and write the code to answer them.

## Cleaning

### Writing a cleaning function

We're going to clean up and extract a few columns from some bank failure data ([source](https://catalog.data.gov/dataset/fdic-failed-bank-list)) that lives in `../data/banklist.csv`.

To keep things tidy, we're going to write a function whose job is to clean up a single row of data. Our function, `cleanRow`, will accept a dictionary that represents a row of data, clean it up and hand it back to the script.

Here are the fields we care about (the ones that need some cleaning are in bold):

- **Bank Name**: Sometimes has extra whitespace, needs to be uppercase, maybe our house style dictates that ampersands have to be replaced by the word "and"
- **City**: Uppercase it!
- ST
- **Acquiring Institution**: Sometimes has extra whitespace, needs to be uppercase, maybe our house style dictates that ampersands have to be replaced by the word "and"
- **Closing Date**: is in M/D/YYYY format; maybe we need to eventually load this into a database that's expecting YYYY-MM-DD

**Write a function, `cleanRow`, that takes as its only argument a row of data (a dictionary), cleans it up according to the parameters we specified, then returns a clean dictionary to the script. Remember: You can chain string functions together.**

In [ ]:
def cleanRow(row):
    row['Bank Name'] = row['Bank Name'].strip().upper().replace('&', 'AND')
    row['Acquiring Institution'] = row['Acquiring Institution'].strip().upper().replace('&', 'AND')
    row['City'] = row['City'].strip().upper()
    
    closing_date_list = row['Closing Date'].split('/')
    row['Closing Date'] = '-'.join([closing_date_list[2],
                                    closing_date_list[0].zfill(2),
                                    closing_date_list[1].zfill(2)])
    
    return row    

### Writing out clean data to a new file

Sometimes your goal is simple: Take some dirty data, clean it up and write the results to a new file. You can open multiple files in the same `with` block, which makes it easy to loop over rows of dirty data and write out clean data to a new file.

For example (assuming you had a function called `cleanRow`):

```python
with open('mlb-original.csv', 'r', encoding='utf-8') as infile, open('mlb-clean.csv', 'w', encoding='utf-8') as outfile:
    headers = ['Name', 'Position', 'Team', 'Salary']

    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=headers)
    
    writer.writeheader()
    
    for row in reader:
        clean_row = cleanRow(row)
        writer.writerow(clean_row)
```

**Loop through the file of failed banks, calling `cleanRow` on each row of data and writing out to a new file, `banklist-clean.csv`. The clean file only needs these columns: 'Bank', 'Acquiring Institution', 'City', 'ST'.**

In [ ]:
import csv

with open('../data/banklist.csv', 'r', encoding='utf-8') as infile, open('banklist-clean.csv', 'w', encoding='utf-8') as outfile:
    headers = ['bank', 'acquiring_institution', 'closing_date', 'city', 'state']
    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=headers)
    
    writer.writeheader()
    
    for row in reader:
        clean_row = cleanRow(row)
        writer.writerow({'bank': clean_row['Bank Name'],
                         'acquiring_institution': clean_row['Acquiring Institution'],
                         'closing_date': clean_row['Closing Date'],
                         'city': clean_row['City'],
                         'state': clean_row['ST']})    

## Basic analysis

Let's change gears here and look at a recent snapshot of outcome data for animals that left the shelter system in Austin, Texas. The file lives in `../data/animal-data.csv`.

### Exploring your data

Whether I'm using Excel or SQL or code, when I get data, I start by poking around a bit to see what's there. [`csvkit`](https://csvkit.readthedocs.io/en/1.0.2/) is a great command-line tool for doing just that. If you're already knee-deep in a Python script, you can use some built-in tools to get a lay of the land.

I like to use [`Counter`](https://docs.python.org/3/library/collections.html#counter-objects) objects to get a quick sense of what I'm looking at in each field. Maybe we want to start by looking at the 20 most common dog breeds in our data. We might do something like this:

```python
import csv
from collections import Counter

with open('../data/animal-data.csv', 'r', encoding='utf-8') as data_file:
    reader = csv.DictReader(data_file)
    
    c = Counter()

    for row in reader:
        if row['Animal Type'].upper() == 'DOG':
        # ^^^^^^
        # look at us using an if statement to filter data!
        # string comparison is case sensitive, so we're upcasing the text of that value to compare
        # N.B. this does ~not~ change the data in the underlying file -- we're just
        # operating on a copy made in your computer's memory
            
            breed = row['Breed']
            
            # add to counter object
            c[breed] += 1

    # https://docs.python.org/3/library/collections.html#collections.Counter.most_common
    # which returns a list of tuples -- another kind of data structure that you can access
    # using bracket notation
    
    for breeds in c.most_common(20):
        print(breeds[0], '=>', breeds[1])
```

**What animals are in the data besides cats and dogs? Print counts of each type of animal.**

**Extra credit: While you're doing that, print the 10 most common cat names. (Cleaning tip: some names are preceded with an asterisk.)**

**Extra credit 2: While you're doing that, print counts of all outcomes.**

In [ ]:
from collections import Counter

with open('../data/animal-data.csv', 'r', encoding='utf-8') as data_file:
    reader = csv.DictReader(data_file)
    
    animal_type_counter = Counter()
    cat_name_counter = Counter()
    outcome_counter = Counter()
    
    for row in reader:

        animal_type = row['Animal Type']
        animal_type_counter[animal_type] += 1
        
        outcome = row['Outcome Type']
        outcome_counter[outcome] += 1
        
        if row['Animal Type'].upper() == 'CAT' and row['Name']:
            name = row['Name'].replace('*', '')
            cat_name_counter[name] += 1

    print('Animal types:')
    for animal_type in animal_type_counter.most_common():
        print(animal_type[0], '=>', animal_type[1])
    
    print('')
    
    print('10 most common cat names:')
    for name in cat_name_counter.most_common(10):
        print(name[0], '=>', name[1])

    print('')
        
    print('Outcomes:')
    for outcome in outcome_counter.most_common():
        print(outcome[0], '=>', outcome[1])

Maybe you want to look at a field to see how dirty it is -- you want a list of unique values in the field sorted alphabetically so you can scan them. There are a couple of ways to do this in Python. You could use call the `most_common()` method of a Counter object without specifying how many you want. Or, if you don't care about counts, you could use `sorted()` with `set()`:

```python
with open('../data/animal-data.csv', 'r', encoding='utf-8') as data_file:
    reader = csv.DictReader(data_file)
    
    dog_breeds = []

    for row in reader:
        if row['Animal Type'].upper() == 'DOG':
            breed = row['Breed']
            dog_breeds.append(breed)
    
    for breed in sorted(set(dog_breeds)):
        print(breed)
```

If there were misspellings -- maybe you see "Afghan Hound," "Afhgan Hound" and "Afgan Hound" -- then you'd know to add those fixes to your cleaning function.

In [ ]:
with open('../data/animal-data.csv', 'r', encoding='utf-8') as data_file:
    reader = csv.DictReader(data_file)
    
    dog_breeds = []

    for row in reader:
        if row['Animal Type'].upper() == 'DOG':
            breed = row['Breed']
            dog_breeds.append(breed)
    
    for breed in sorted(set(dog_breeds)):
        print(breed)

Maybe you're working on a story about pit bulls and you need to answer these questions:
- How many pit bulls or pit bull mixes (any dog with 'pit bull' in its breed) have left the shelter system?
- What percentage does this represent of dogs that left the shelter system?

**Print a sentence that answers these questions.**

Keep in mind: 
- You can use `if` to filter data
- You can use the `in` operator to see if one string exists in another string
- You can set up a counting variable manually -- e.g., `all_dogs = 0` -- before you begin your loop, then add to it with each loop iteration
- Depending on the level of decimal precision you need, you could use `format` or `round` to print the percentage

In [ ]:
with open('../data/animal-data.csv', 'r', encoding='utf-8') as data_file:
    reader = csv.DictReader(data_file)
    
    all_dogs = 0
    pit_bulls = 0

    for row in reader:
        if row['Animal Type'].upper() == 'DOG':
            all_dogs += 1
            if 'PIT BULL' in row['Breed'].upper():
                pit_bulls += 1
    
    rough_pct = round((pit_bulls / all_dogs) * 100)
    summary = 'Of the {:,} dogs that left the shelter system,' \
              ' about {} percent ({:,}) were pit bulls or pit' \
              ' bull mixes.'.format(all_dogs, rough_pct, pit_bulls)
            
    print(summary)

### Basic aggregations

Let's jump back over to our baseball data in `../data/mlb.csv`.

The data show the 2016 MLB opening-day roster with salaries. Maybe you want to check out some aggregate stats: player count, total payroll, average and median salary.

Later versions of Python, such as the one we're using, have a module called `statistics` we can use. First, we need to extract the salaries from each row of data into a separate list. That way we can get the player count with `len()`, the total with `sum()` and the mean/median with a couple of `statistics` methods.

To do that, we could write some code that looks like this:

```python
import csv
import statistics

with open('../data/mlb.csv', 'r', encoding='utf-8') as infile:
    salaries = []

    for row in reader:

        # call the `float` function to turn the string into a number that we can do math on
        pay = float(row['SALARY'])
        salaries.append(pay)

    total = sum(salaries)
    # etc. ...
```

... but there's a quicker, easier way. You can use a [list comprehension](http://www.pythonforbeginners.com/basics/list-comprehensions-in-python):

```python
import csv
import statistics

with open('../data/mlb.csv', 'r', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    salaries = [float(row['SALARY']) for row in reader]
    # ^^^ this is the list comprehension ^^^
    # isn't this insane can you even believe it
    # yeah me neither

    count = len(salaries)
    total = sum(salaries)
    average = statistics.mean(salaries)
    median = statistics.median(salaries)
    
    summary = 'MLB opening day roster 2016 stats:\n- Players: {}\n' \
              '- Total payroll: {}\n- Average: {}\n' \
              '- Median: {}'.format(count, total, average, median)

    print(summary)
```

Note how I'm breaking a string over multiple lines with a backslash. Note, too, that the overflow lines match up with where the string starts on the first line. Whitespace, man.

**Your turn! Find the sum, count, average and median for your favorite baseball team, or for the Arizona Diamondbacks if you don't have a favorite team. Protip: You can add an `if` statement to a list comprehension (!)**

**Extra credit: What's the median pay for catchers?**

In [ ]:
import csv
import statistics

with open('../data/mlb.csv', 'r', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    az_salaries = [float(row['SALARY']) for row in reader if row['Team'] == 'Arizona Diamondbacks']

    count = len(az_salaries)
    total = sum(az_salaries)
    average = statistics.mean(az_salaries)
    median = statistics.median(az_salaries)

    
    summary = 'Arizona Diamondbacks 2016 roster stats:\n- Players: {}\n' \
              '- Total payroll: {}\n- Average: {}\n' \
              '- Median: {}'.format(count, total, average, median)

    print(summary)
    
    # print a blank line for space
    print('')
    
    # now let's check out catchers
    
    # first we need to go back to the beginning of the file using `seek`
    infile.seek(0)
    
    # and re-set the reader
    reader = csv.DictReader(infile)
    
    catcher_salaries = [float(row['SALARY']) for row in reader if row['POS'] == 'C']

    catcher_median = statistics.median(catcher_salaries)
    
    print("Catcher median:", catcher_median)